# Glubteba
> Investigating this elusive GO loader

- toc: true 
- badges: true
- categories: [Glubteba,loader,triage]

## Overview

Taking a look at some random GO malware with light obfuscation. Possibly linked to Glubteba.

According to [Sophos](https://news.sophos.com/wp-content/uploads/2020/06/glupteba_final-1.pdf) there are 3 possible GO components linked to a Glupteba infection.

- `payload64.dll` - a 64bit payload used along with the EternalBlue exploit to download the main Glupteba component
- `payload32.dll` - a 32bit payload used along with the EternalBlue exploit to download the main Glupteba component
- `collectchromefingerprint.exe` - a UPX packed binary used to locate Chrome and connect to `http[:]//swebgames[.]site/test.php?uuid=%s&browser=chrome` to register to browser-based fingerprinting services


## References 

- [Glupteba: Hidden Malware Delivery in Plain Sight (pdf)](https://news.sophos.com/wp-content/uploads/2020/06/glupteba_final-1.pdf)
- [Disrupting the Glupteba operation](https://blog.google/threat-analysis-group/disrupting-glupteba-operation/)
- [Tracking Malicious Glupteba Activity Through the Blockchain](https://www.nozominetworks.com/blog/tracking-malicious-glupteba-activity-through-the-blockchain/)


### Sample
- [3cc7fb757318a924954642bfa36dda9c2cf53c9446a85bdcda756603e17a6961](https://www.unpac.me/results/8674879c-122d-4973-b45a-22a0f8217d95?hash=3cc7fb757318a924954642bfa36dda9c2cf53c9446a85bdcda756603e17a6961#/)
- not glupteba but possibly similar obfuscator [dd124a7b396150e4d8275c473594e47ac24606ef0955e2c13310aac9045554ac](https://www.unpac.me/results/c2d49f20-7bd7-48af-b5f6-4ead73d3790f?hash=dd124a7b396150e4d8275c473594e47ac24606ef0955e2c13310aac9045554ac#/)

## Analysis
Looking through the Glupteba IOCs [released by Sophos](https://github.com/sophoslabs/IoCs/blob/master/Trojan-Glupteba) we can see two hashes labeled as `app.exe` which is the main component of Glupteba according to their report.

- `407c70f0c1a1e34503dae74dd973cf037d607e3c4deb8f063d33f2142f1baf71` [UnpacMe](https://www.unpac.me/results/9754bb3a-2e48-4e8d-b3f0-b8d040bf7a43?hash=407c70f0c1a1e34503dae74dd973cf037d607e3c4deb8f063d33f2142f1baf71#/)
- `83bbe9e7b7967ecbc493f8ea40947184c6c7346c6084431fceea0401a6279d29` [UnpacMe](https://www.unpac.me/results/d6e7585a-69e5-4946-9e22-f6846b9c1c13?hash=83bbe9e7b7967ecbc493f8ea40947184c6c7346c6084431fceea0401a6279d29#/)

Upon analysis, both of these appear to be the 32-bit version of the payload delivered by the EternalBlue SMB exploit `payload32.dll`. One packed with UPX and the other not packed. This lead to some confusion as it is unclear in the report if this is considered the main Glubteba component. 

Comparing these samples with our sample `3cc7fb757318a924954642bfa36dda9c2cf53c9446a85bdcda756603e17a6961` we can see some overlap in functionality but our sample contains significantly more features including anti-analysis checks and a TOR browser and appears to better match with the full Glupteba functionally listed in the Sophos report. It is unclear if this module was part of the original Glupteba build analyzed by Sophos in 2020 or if it is a new build however it appears to now be the main Glupteba module. 


## Yara
```c
rule glub_hunt {
    strings:

        $report1 = "main.reportInstallFailure"
        $report2 = "main.sendLog"
        $report3 = "main.sendLogError"
        
        $anti1 = "main.isRunningInsideVirtualBox"
        $anti2 = "main.isRunningInsideVMWare"
        $anti3 = "main.isRunningInsideParallels"
        $anti4 = "main.isRunningInsideVirtualPC"
        $anti5 = "main.isRunningInsideXen"
        $anti6 = "main.isRunningInsideAnyRun"
        $anti7 = "main.isRunningInsideVM"
        
        $smb1 = "main.handleConnSMB"
        $smb2 = "main.handleConnSMB.func1"
        $smb3 = "main.listenAndServeSMB"
        $smb4 = "main.listenAndServeSMB.func2"
        $smb5 = "main.listenAndServeSMB.func1"
        $smb6 = "main.watchSMB"
        
        $tor1 = "main.watchTor"
        $tor2 = "main.watchTor.func1"
        $tor3 = "main.watchTor.func2"
        
        $drv1 = "main.createDevice"
        $drv2 = "main.excludeFilename"
        $drv3 = "main.excludeFilename.func1"
        $drv4 = "main.disablePatchGuard"
        $drv5 = "main.disablePatchGuard.func1"

    condition:
        filesize > 7MB and
        filesize < 10MB and
        5 of them
}
```